
# Sauk-Suiattle Sediment Model

This data is compiled to digitally observe the Skagit Watershed, powered by HydroShare. <br />
<img src= "http://www.sauk-suiattle.com/images/Elliott.jpg"
style="float:left;width:175px;padding:20px">   
<br />
This is a beta version of a computational network-based sediment model was developed in order to connect processes of sediment supply on hillslopes, routing in streams, and deposition in reservoirs. The sediment model is developed in a framework called Landlab and driven by a physically-based, distributed hydrology model called DHSVM. The coupled sediment-hydrology model is designed to integrate relevant temporal and spatial scales of hillslope geomorphology, hydroclimatology and river network processes along with answering questions that are relevant to engineering application. The coupled model framework is designed to be applicable in other global watersheds, and could be useful for predicting sediment budgets particularly in the face of environmental and land use/land cover changes. 
<br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
<br />
#### A Watershed Dynamics Model by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington 

[Erkan To DO - -  add citations to Claire's AGU presentations and DOE report, Landlab paper, DHSVM-PNNL repo, others?]

## To run this notebook:

Click in each shaded cell below and "shift + enter" to run each code block. Alternatively, you can run groups of cells by clicking "Cell" on the menu above and selecting your run options from the pull-down menu. This is also where you can clear outputs from previous runs.

If an error occurs, click on *Kernal* and *Restart and Clear Outputs* in the menu above.

## 1. Introduction
Sediment discharge, its frequency and magnitude, and the size distribution of sediment load vary in space and time in a river. Sediment input into channels can be dominated by a single source such as a stratovolcano (e.g., Glacier Peak), or can be from distributed sources on hillslopes and zero-order basins, as well as from the banks of the channel itself through bank erosion.  Sediment is either transported within the flow as suspended load or transported along the riverbed as bedload. Suspended sediment includes sediments that are transported in suspension as a result of flow shear stress. It is composed of wash load (mostly clay and silt that is more or less continually in suspension) and suspension from bedload during higher flow; bed material in suspension includes coarse silt and sand that may move as bed load during lower flow. Bedload is generally assumed to represent 10% (or more generally in the 5% to 20% range) of the total sediment load. This percentage also depends on the location in the channel network and the type of watershed soil type. The suspended sediment load varies with discharge, often increasing nonlinearly as discharge rises because the greater turbulence associated with higher discharge allows a greater load of sediment to be held in suspension. 

<br /> <img src="https://www.hydroshare.org/resource/d5f1f96849554e03aa98bed5aff36cf7/data/contents/SedimentConcentrationbyVolume_Jakob_Jordon_2001.JPG" style="float:center;width:700px;padding:40px">

Figure 1. Classification of sediment flow processes with respect to sediment concentration, velocity, and process domains (modified from Jakob and Jordan, 2001).

Two related questions of interest for predicting erosion rates and sediment yields in the Puget Sound region are:  

1)	Do rivers that drain stratovolcanos and retreating glaciers exhibit larger erosion rates, and how modern and long-term erosion rates compare? 

2)	Can a single suspendend sediment transport relationship predict loads across Puget Sound basins?

In stratovolcanoes in high Alpine areas like Glacier Peak, a large portion of the loose pyroclastic deposits (clastic rock formed by volcanic explosion or ejected from a volcanic vent) are eroded by glaciers. Currently there are over a dozen glaciers on the sides of this volcano. In the model developed in this Notebook, we highlight network sediment results from seven glaciers; five in the Headwater Suiattle River (HUC 171100060201) including Dusty, Chocolate, Cool, Suiattle, and Honeycomb Glaciers and two in the Miners Creek Suiattle River (HUC 171100060202) including Vista and Ermine Glaciers. 

## 2.0 Methods

### 2.1. Install Python packages and libraries 
The CUAHSI JupyterHub server provides many Python packages and libraries, but to add additional libraries to your personal user space, use the cell below.  To request an Installation to the server, visit https://github.com/hydroshare/hydroshare-jupyterhub, create a New Issue, and add the label 'Installation Request'. Uncomment the lines below to install the library. Python2 and Python3 kernels are both available. If you are a new Landlab user on Hydroshare you are advised to run the code block below in your first run of this tutorial, then comment them out.

In [1]:
#%% Import modules and define functions
import os
import csv
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.stats

#Import utilities for importing and exporting to HydroShare
from utilities import hydroshare
hs=hydroshare.hydroshare()
import ogh
# path and directory management# path  
import os



Adding the following system variables:
   HS_USR_NAME = ChristinaBandaragoda
   HS_RES_ID = d5f1f96849554e03aa98bed5aff36cf7
   HS_RES_TYPE = compositeresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => ChristinaBandaragoda

The hs_utils library requires a secure connection to your HydroShare account.
Enter the HydroShare password for user 'ChristinaBandaragoda': ········
Successfully established a connection with HydroShare


In [2]:
hs=hydroshare.hydroshare()
homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))
print('Data will be loaded from and saved to:'+homedir)

Adding the following system variables:
   HS_USR_NAME = ChristinaBandaragoda
   HS_RES_ID = d5f1f96849554e03aa98bed5aff36cf7
   HS_RES_TYPE = compositeresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => ChristinaBandaragoda
Successfully established a connection with HydroShare
Data will be loaded from and saved to:/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


In [3]:
#homedir = hs.getContentPath(os.environ["HS_RES_ID"])
homedir = '/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents'
os.chdir(homedir)
print('Data will be loaded from and save to:'+homedir)

Data will be loaded from and save to:/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


### 2.2 Import DHSVM network files and modeled streamflow 

In [4]:
# GIS file developed from https://github.com/pnnl/DHSVM-PNNL/tree/master/CreateStreamNetwork_PythonV
# Note for Zhuoron: This code needs it's own citation and reference
network_input = os.path.join(homedir, 'streamfile_2km2_sauk.txt')
# DHSVM model inputs
networkdat_input='stream.network.dat'
map_input='stream.map.dat'

#### Run following cells to import extract and copy DHSVM temperature model Outflow.Only files on HydroShare

In [ ]:
#hs.getResourceFromHydroShare('f16bdb504c6a4fb39e80ff1070a86704')

In [15]:
#!gunzip /home/jovyan/work/notebooks/data/f16bdb504c6a4fb39e80ff1070a86704/f16bdb504c6a4fb39e80ff1070a86704/data/contents/*/*

In [16]:
#!cp /home/jovyan/work/notebooks/data/f16bdb504c6a4fb39e80ff1070a86704/f16bdb504c6a4fb39e80ff1070a86704/data/contents/Sauk_1969-2001_biasLivneh_WRF_TH8-WhiteTH15_mu12.8_his/Outflow.Only .

In [4]:
# DHSVM model output  - flow at each stream node
#streamflow_input='Outflow.Only'

## Make an array from modeled streamflow spreadsheets

In [12]:
forcing_start_date=datetime.date(1961,1,1)
forcing_end_date=datetime.date(2099,9,30)
time0=pd.date_range(start=forcing_start_date, end=forcing_end_date, freq='D')


forcing_start_date=datetime.date(1961,1,1)
forcing_end_date=datetime.date(2010,12,31)
time1=pd.date_range(start=forcing_start_date, end=forcing_end_date, freq='D')

forcing_start_date=datetime.date(2000,1,1)
forcing_end_date=datetime.date(2050,12,31)
time2=pd.date_range(start=forcing_start_date, end=forcing_end_date, freq='D')

forcing_start_date=datetime.date(2025,1,1)
forcing_end_date=datetime.date(2075,12,31)
time3=pd.date_range(start=forcing_start_date, end=forcing_end_date, freq='D')

forcing_start_date=datetime.date(2050,1,1)
forcing_end_date=datetime.date(2099,9,30)
time4=pd.date_range(start=forcing_start_date, end=forcing_end_date, freq='D')


In [18]:
#Details from DHSVM Network model input
#    1  57     0.00107         750.00000   6      -1 SAVE "outlet"
#   19  53     0.00237        6368.37662  12       5 SAVE "12189500"
#  187  30     0.00546        4945.58441  12     147 SAVE "12187500"
#  224   7     0.01904        1722.79221  18     207 SAVE "12187000"
#  257  18     0.02688        1536.39610  18     255 SAVE "12186500"
        
#DHSVM network link
outlet=1
middlesauk=187   #Sauk River near Darrington NWISDV:12187500
lowersauk_dhsvmid=19  #Sauk River near Sauk NWISDV:12189500  PNNL-Python code link 20; (AML code link 19)
lowersauk_pnnl_area=20

#DHSVM network link - other locations
suiattleglacier=368
SaukaboveWhiteChuck=279  
SaukWhiteChuck=263
headwaters=260


In [19]:
DHSVMnetwork=pd.read_table(network_input)
area_o=DHSVMnetwork.total_ca_mean[outlet-1]/1000000
area_low=DHSVMnetwork.total_ca_mean[lowersauk_pnnl_area-1]/1000000
area_mid=DHSVMnetwork.total_ca_mean[middlesauk-1]/1000000
print(round(area_o,0))
print(round(area_low,0))
print(round(area_mid,0))

1905.0
1866.0
772.0


In [70]:
def suspsedtransport(df,time,b0,b1,bcf,area,location):
    df['Date']=pd.to_datetime(df.Year.astype(str) + '-' + df.Month.astype(str) + '-' + df.Day.astype(str))
    df=df.set_index('Date')
    df_cms=df.loc[time].drop(['Year','Month','Day'],axis=1)*0.028316846592 
    s=(b0*df_cms**b1)*bcf   #tons per day
    total_suspsed_T=s.cumsum(axis=0).iloc[-1:] #Total tons in period
    total_suspsed_A=total_suspsed_T/(len(time)/365) #Total tons/year
    total_suspsed_Y=total_suspsed_A/area  #mean annual tons/sqkm
    Q_max=pd.DataFrame(df_cms.max(axis=0)).T
    Q_mean=pd.DataFrame(df_cms.mean(axis=0)).T
    #print(location)
    #print(total_suspsed, 'sediment tons per day')
    #print(area, 'total drainage area m2')
    #print(round(Q_max,0), 'peak flow event in record (cms)')
    
    return(total_suspsed_T,total_suspsed_A,total_suspsed_Y,df_cms,Q_max,Q_mean)


## Run the function for Lower Sauk Bias Corrected

In [71]:
xls = pd.ExcelFile('SaukRiver_nrSauk_draft_BC.xls')
df_bc45 = pd.read_excel(xls, 'daily_cfs_rcp45')
df_bc85 = pd.read_excel(xls, 'daily_cfs_rcp85')

b0_ls=0.0000025 
b1_ls=3.8
lowsauk_sed_bcf=1.08
lowsauk_location='Lower Sauk - USGS Sauk near Sauk (12189500)  RCP45 '


lowsauk_t_bc=pd.DataFrame()
lowsauk_a_bc=pd.DataFrame()
lowsauk_y_bc=pd.DataFrame()
lowsauk_q_bc=pd.DataFrame()
lowsauk_qmax_bc=pd.DataFrame()
lowsauk_qmean_bc=pd.DataFrame()

for df in [df_bc45,df_bc85]:

    for time in [time1,time2,time3,time4]:
        T,A,Y,q,qmax,qmean=suspsedtransport(df=df,time=time,b0=b0_ls,b1=b1_ls,bcf=lowsauk_sed_bcf,
                                  area=area_low,location=lowsauk_location)
        lowsauk_t_bc=pd.concat([lowsauk_t_bc,T])
        lowsauk_a_bc=pd.concat([lowsauk_a_bc,A])
        lowsauk_y_bc=pd.concat([lowsauk_y_bc,Y])
        lowsauk_q_bc=pd.concat([lowsauk_q_bc,q])
        lowsauk_qmax_bc=pd.concat([lowsauk_qmax_bc,qmax])   
        lowsauk_qmean_bc=pd.concat([lowsauk_qmean_bc,qmean])   
        
lowsauk_t_bc.to_csv('lowsauk_t_bc.csv',header=True)
lowsauk_a_bc.to_csv('lowsauk_a_bc.csv',header=True)
lowsauk_y_bc.to_csv('lowsauk_y_bc.csv',header=True)
lowsauk_q_bc.to_csv('lowsauk_q_bc.csv',header=True)
lowsauk_qmax_bc.to_csv('lowsauk_qmax_bc.csv',header=True)
lowsauk_qmean_bc.to_csv('lowsauk_qmean_bc.csv',header=True)

In [72]:
lowsauk_qmean_bc

,bcc-csm1-1-m,CanESM2,CSIRO-Mk3-6-0,CCSM4,CNRM-CM5,HadGEM2-CC365,HadGEM2-ES365,IPSL-CM5A-MR,NorESM1-M,MIROC5
0,123.789332,123.854114,123.846674,122.709780,123.817483,123.850282,123.865572,123.854941,123.861746,123.941780
0,124.532751,124.269255,125.244644,109.462359,122.255445,132.284492,121.871839,122.263157,129.170375,124.955940
0,127.152082,126.690309,130.386925,111.656964,125.469348,128.905842,127.032060,125.887681,137.779625,130.836219
0,127.049725,129.361130,130.525681,113.292094,129.413462,129.689611,130.361445,130.670694,140.717485,135.445568
0,123.802584,123.871851,123.809097,122.748516,123.827411,123.824948,123.895998,123.854179,123.884211,123.863435
0,129.151687,125.244944,123.564436,108.788777,121.631384,125.593342,124.595772,128.682738,129.802503,135.347625
0,131.491708,129.280598,126.763747,105.225494,125.570973,127.093583,130.091196,137.045310,144.610671,131.476916
0,132.709383,127.174886,128.601567,111.090644,130.163413,131.219868,123.518062,134.947429,143.458434,135.526383


In [73]:
lowsauk_qmax_bc

,bcc-csm1-1-m,CanESM2,CSIRO-Mk3-6-0,CCSM4,CNRM-CM5,HadGEM2-CC365,HadGEM2-ES365,IPSL-CM5A-MR,NorESM1-M,MIROC5
0,1628.726712,1628.726712,1628.726712,2462.276359,1628.726712,1628.726712,1628.726712,1628.726712,1628.726712,1628.726712
0,2180.518639,1873.242795,1978.243673,2669.250817,1989.011154,1539.869891,2130.694269,2731.663582,1926.149736,3200.856032
0,1365.367437,2051.686274,1978.243673,3589.735279,2037.099559,2009.988840,2130.694269,3672.425993,2098.383643,3200.856032
0,1914.465045,2051.686274,1982.521272,3589.735279,2037.099559,2713.290168,2093.528408,3672.425993,2098.383643,2196.979788
0,1628.726712,1628.726712,1628.726712,2462.276359,1628.726712,1628.726712,1628.726712,1628.726712,1628.726712,1628.726712
0,1523.932717,1493.376178,1592.232498,1974.101428,1611.917350,2727.385530,1992.891213,2208.015627,1493.376178,1635.993211
0,3033.490641,1515.580012,1592.232498,1974.101428,1611.917350,2727.385530,2392.810264,2208.015627,1655.808122,2181.300240
0,3033.490641,1782.857092,1853.017601,2483.971481,2265.427156,2714.427911,2392.810264,2139.978882,1851.006567,2458.260011


## Run the function for Lower Sauk

In [74]:
xls = pd.ExcelFile('SaukRiver_nrSauk_SC2_future.xls')
df_bc45 = pd.read_excel(xls, 'daily_cfs_rcp45')
df_bc85 = pd.read_excel(xls, 'daily_cfs_rcp85')

b0_ls=0.0000025 
b1_ls=3.8
lowsauk_sed_bcf=1.08
lowsauk_location='Lower Sauk - USGS Sauk near Sauk (12189500)  RCP45 '

lowsauk_t=pd.DataFrame()
lowsauk_a=pd.DataFrame()
lowsauk_y=pd.DataFrame()
lowsauk_q=pd.DataFrame()
lowsauk_qmax=pd.DataFrame()
lowsauk_qmean=pd.DataFrame()

for df in [df_bc45,df_bc85]:

    for time in [time1,time2,time3,time4]:
        T,A,Y,q,qmax,qmean=suspsedtransport(df=df,time=time,b0=b0_ls,b1=b1_ls,bcf=lowsauk_sed_bcf,
                                  area=area_low,location=lowsauk_location)
        lowsauk_t=pd.concat([lowsauk_t,T])
        lowsauk_a=pd.concat([lowsauk_a,A])
        lowsauk_y=pd.concat([lowsauk_y,Y])
        lowsauk_q=pd.concat([lowsauk_q,q])
        lowsauk_qmax=pd.concat([lowsauk_qmax,qmax])   
        lowsauk_qmean=pd.concat([lowsauk_qmean,qmean]) 
        
lowsauk_t.to_csv('lowsauk_t.csv',header=True)
lowsauk_a.to_csv('lowsauk_a.csv',header=True)
lowsauk_y.to_csv('lowsauk_y.csv',header=True)
lowsauk_q.to_csv('lowsauk_q.csv',header=True)
lowsauk_qmax.to_csv('lowsauk_qmax.csv',header=True)
lowsauk_qmean.to_csv('lowsauk_qmean.csv',header=True)

## Run the function for Middle Sauk

In [78]:
xls = pd.ExcelFile('SaukRiver_Darrington_SC2_future.xls')
df_bc45 = pd.read_excel(xls, 'daily_cfs_rcp45')
df_bc85 = pd.read_excel(xls, 'daily_cfs_rcp85')

b0_ms=0.0000757 
b1_ms=3.22
bcf_ms = 1.25

midsauk_location='Sauk - USGS Sauk near Darrington (12187500)  '

midsauk_t=pd.DataFrame()
midsauk_a=pd.DataFrame()
midsauk_y=pd.DataFrame()
midsauk_q=pd.DataFrame()
midsauk_qmax=pd.DataFrame()
midsauk_qmean=pd.DataFrame()

for df in [df_bc45,df_bc85]:

    for time in [time1,time2,time3,time4]:
        T,A,Y,q,qmax,qmean=suspsedtransport(df=df,time=time,b0=b0_ms,b1=b1_ms,bcf=bcf_ms,
                                  area=area_mid,location=midsauk_location)
        midsauk_t=pd.concat([midsauk_t,T])
        midsauk_a=pd.concat([midsauk_a,A])
        midsauk_y=pd.concat([midsauk_y,Y])
        midsauk_q=pd.concat([midsauk_q,q])
        midsauk_qmax=pd.concat([midsauk_qmax,qmax])   
        midsauk_qmean=pd.concat([midsauk_qmean,qmean])  

        
midsauk_t.to_csv('midsauk_t.csv',header=True)
midsauk_a.to_csv('midsauk_a.csv',header=True)
midsauk_y.to_csv('midsauk_y.csv',header=True)
midsauk_q.to_csv('midsauk_q.csv',header=True)
midsauk_qmax.to_csv('midsauk_qmax.csv',header=True)
midsauk_qmean.to_csv('midsauk_qmean.csv',header=True)

In [ ]:
midsauk_y

In [ ]:
midsauk_qmax

In [ ]:
midsauk_qmean

In [85]:
writer = pd.ExcelWriter('SaukSuspendedSediment_09202018.xlsx')

lowsauk_y_bc.to_excel(writer,'lowsauk_y_bc',header=True)
lowsauk_qmax_bc.to_excel(writer,'lowsauk_qmax_bc',header=True)
lowsauk_qmean_bc.to_excel(writer,'lowsauk_qmean_bc',header=True)

lowsauk_y.to_excel(writer,'lowsauk_y',header=True)
lowsauk_qmax.to_excel(writer,'lowsauk_qmax',header=True)
lowsauk_qmean.to_excel(writer,'lowsauk_qmean',header=True)

midsauk_y.to_excel(writer,'midsauk_y',header=True)
midsauk_qmax.to_excel(writer,'midsauk_qmax',header=True)
midsauk_qmean.to_excel(writer,'midsauk_qmean',header=True)

writer.save()

In [83]:
!pwd

/home/jovyan/work/notebooks/data/d5f1f96849554e03aa98bed5aff36cf7/d5f1f96849554e03aa98bed5aff36cf7/data/contents


In [67]:
Sauk_DA=DHSVMnetwork.total_ca_mean[lowersauk]
N_of_days=len(streamflow[lowersauk])

Sum_SF=sum(streamflow[lowersauk][1:-1])
Mean_Annual_Runoff=(((Sum_SF/N_of_days)*86488*365)/Sauk_DA)*1000 #

Sum_Mass_of_SSL=sum(s_ls[1:-1])
Mean_Daily_SSL=Sum_Mass_of_SSL/N_of_days
Mean_Annual_SSL=Mean_Daily_SSL*365

Mean_Annual_SSL_per_area=Mean_Annual_SSL/(Sauk_DA*10**-6)
Mean_Annual_SSL_Vol=Mean_Annual_SSL/2.65

Mean_Annual_SSL_depth=(Mean_Annual_SSL_Vol/Sauk_DA)*1000


print(Mean_Annual_Runoff,'Mean Annual Runoff, [mm]')
print(Mean_Annual_SSL_per_area,'Mean Annual SSL [tons/km2]')
print(Mean_Annual_SSL_depth,'Mean Annual SSL erosion [mm/y] ')
print(Sauk_DA*10**-6,'Sauk Drainage Area km2')

1759.16898788 Mean Annual Runoff, [mm]
128.946263796 Mean Annual SSL [tons/km2]
0.0486589674702 Mean Annual SSL erosion [mm/y] 
1865.587456 Sauk Drainage Area km2


In [57]:
#  Middle Sauk

#Model No. 2.4  
b0_ms=0.0000757 
b1_ms=3.22
bcf_ms = 1.25

#daily streamflow input (cms)
s_ms=(b0_ms*streamflow[middlesauk]**b1_ms)*bcf_ms   #tons per day

print(sum(s_ms[1:-1]), 'sediment tons per day')
print(DHSVMnetwork.total_ca_mean[middlesauk], 'total drainage area m2')
print(max(streamflow[middlesauk]), 'peak flow event in record (cms)')

1184030.65044151 sediment tons per day
1192500 total drainage area m2
465.2274375 peak flow event in record (cms)


In [61]:
Sauk_DA=DHSVMnetwork.total_ca_mean[middlesauk-1]
N_of_days=len(streamflow[middlesauk])

Sum_SF=sum(streamflow[middlesauk][1:-1])
Mean_Annual_Runoff=(((Sum_SF/N_of_days)*86488*365)/Sauk_DA)*1000 #

Sum_Mass_of_SSL=sum(s_ls[1:-1])
Mean_Daily_SSL=Sum_Mass_of_SSL/N_of_days
Mean_Annual_SSL=Mean_Daily_SSL*365

Mean_Annual_SSL_per_area=Mean_Annual_SSL/(Sauk_DA*10**-6)
Mean_Annual_SSL_Vol=Mean_Annual_SSL/2.65

Mean_Annual_SSL_depth=(Mean_Annual_SSL_Vol/Sauk_DA)*1000


print(Mean_Annual_Runoff,'Mean Annual Runoff, [mm]')
print(Mean_Annual_SSL_per_area,'Mean Annual SSL [tons/km2]')
print(Mean_Annual_SSL_depth,'Mean Annual SSL erosion [mm/y] ')
print(Sauk_DA*10**-6,'Sauk Drainage Area km2')

1886.57194771 Mean Annual Runoff, [mm]
519.573651306 Mean Annual SSL [tons/km2]
0.196065528795 Mean Annual SSL erosion [mm/y] 
771.660032 Sauk Drainage Area km2


The information, data, or work presented herein was funded in part by the Office of Energy Efficiency and Renewable Energy (EERE), U.S. Department of Energy, under Award Number DE-EE0006506 and the Hydro Research Foundation. Neither the United States Government nor any agency thereof, nor any of their employees, makes and warranty, express or implied, or assumes and legal liability or responsibility for the accuracy, completeness, or usefulness of any information, apparatus, product, or process disclosed, or represents that its use would not infringe privately owned rights. Reference herein to any specific commercial product, process, or service by trade name, trademark, manufacturer, or otherwise does not necessarily constitute or imply its endorsement, recommendation or favoring by the United States Government or any agency thereof. The views and opinions of authors expressed herein do not necessarily state or reflect those of the United States Government or any agency thereof.

[Disclaimer like this needed for BIA funding]

[Christina TO DO - add save back to HydroShare end bits]